## Question 14

In [ ]:
using LinearAlgebra

function structured_P(L::Int; p::Float64 = 0.45, r::Float64 = 0.01)::Matrix{Float64}
    q = 1 - p - r
    P = diagm(fill(r,L)) + diagm(-1=>fill(q,L-1)) + diagm(1 => fill(p,L-1))
    P[1,1] = 1-p
    P[L,L] = 1-q
    return P
end

structured_π(L::Int; p::Float64 = 0.45, r::Float64 = 0.01)::Vector{Float64} = begin
    q = 1 - p - r
    [(p/q)^i  for i in 1:L] * (q-p) / p / (1-(p/q)^L) #Explicit expression (birth death)
end;

P = structured_P(5)
π = structured_π(5)
@show sum(π)
@show sum(π) ≈ 1
π

In [ ]:
function get_π_nullspace(P)
    #we wish to solve the equation πP - π = 0
    #i.e. we want the nullspace of (P-I)'
    basis = nullspace((P-I)')
    return basis/sum(basis)
end

get_π_nullspace(structured_P(5))

In [ ]:
function get_π_limit(P,n=100)
    return (P^n)[1,:]
end

get_π_limit(structured_P(5))

In [ ]:
function get_π_eigen(P)
    F = eigen(P')
    π = F.vectors[:,end]
    return π/sum(π)
end

get_π_eigen(structured_P(5))

In [ ]:
using StatsBase

function get_π_stats(P,sample_size=10^6)
    n = length(P[1,:])
    π = zeros(n)
    x = 1
    for i in 1:sample_size
        x = wsample(1:n,P[x,:])
        π[x] += 1
    end
    return π/sample_size
end

get_π_stats(structured_P(5))


In [ ]:
using BenchmarkTools
using StatsPlots
function evaluate()
    results = []
    for n in [2,3,4,5,10,20,30,40,50,100,200,300,400,500,1000]
        P = structured_P(n)
        π = structured_π(n)
        π1 = get_π_nullspace(P)
        π2 = get_π_eigen(P)
        π3 = get_π_limit(P,300)
        π4 = get_π_stats(P)
        results = push!(results,[norm(π-π1),norm(π-π2),norm(π-π3),norm(π-π4)])
    end
    return results
end

res = evaluate()
x = [2,3,4,5,10,20,30,40,50,100,200,300,400,500,1000]
data = collect(zip(res...))
y = hcat(collect.(data)...)
groupedbar(string.(x),y,yaxis=:log,xlabel="dimension",ylabel="error",label=hcat("nullspace","eigen","limit","stats"))
png("markov_accuracy.png")

![]("markov_accuracy.png")

In terms of accuracy, computing the eigenvector corresponding to eigenvalue 1 is the most consistently accurate for higher dimensions. Computing the nullspace is marginally better at lower dimensions, but performs worse with higher dimensions. Manually computing the limit is reasonable for low dimensions, but is not precise enough for higher dimensions, at least with only 300 iterations. Computing a random Markov chain has relatively consistent low accuracy, but this could be improved by increasing the number of samples (at the expense of compute time).